# Set partitioning

Given a list of numbers, partition the values into two sets of equal sum.

In [2]:
values = [1, 2, 3, 4, 5, 6, 7, 8]

# Create the BQM object

- Use one binary variable x for each number. If x = 1, the value belongs to one set, otherwise it belong to antoher.
- There is no object, only a constraint
- The sum of the values in each set must be equal

$$ \sum_i v_i x_i = \sum_i v_i (1 - x_i) $$

After simplifying:

$$ \sum_i 2 v_i x_i - \sum_i v_i = 0$$

In [3]:
from dimod import AdjVectorBQM

bqm = AdjVectorBQM('BINARY')
n = len(values)
x = {i: bqm.add_variable(f'x_{i}') for i in range(n)}
    
bqm.add_linear_equality_constraint(
    [(x[i], 2.0 * values[i]) for i in range(n)],
    constant=-sum(values),
    lagrange_multiplier=10
)

In [5]:
from dimod import ExactSolver

response = ExactSolver().sample(bqm).truncate(5)
solution = response.first.sample
print(response)

  x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 energy num_oc.
0   0   1   0   1   1   0   1   0    0.0       1
1   0   0   1   0   0   0   1   1    0.0       1
2   1   1   1   0   1   0   1   0    0.0       1
3   1   0   1   0   0   1   0   1    0.0       1
4   1   0   0   1   0   1   1   0    0.0       1
['BINARY', 5 rows, 5 samples, 8 variables]


In [16]:
set1 = {values[i] for i in x if solution[x[i]]}
set2 = {values[i] for i in x if not solution[x[i]]}
print(f'{sum(set1)} = sum{tuple(set1)}')
print(f'{sum(set2)} = sum{tuple(set2)}')

18 = sum(2, 4, 5, 7)
18 = sum(8, 1, 3, 6)


# Partitioning to more than two sets

- We will need one binary variable for each number and set combination
- The binary value xij = 1 if value i belongs to set j
- Each value can only be assigned to one set

In [2]:
values = [7, 2, 3, 1, 8, 3, 1, 2, 9]
bqm = AdjVectorBQM('BINARY')
n = len(values)
m = 3 # num_partitions

x = {(i, k): bqm.add_variable((f'x_{i}', k)) 
     for i in range(n)
     for k in range(m)
    }

NameError: name 'AdjVectorBQM' is not defined

# No objective, only constraints

For each pair of sets, ensure that the sum of the two sets is equal

$$ \sum_i v_i x_{ij} = \sum_i v_i x_{ik} $$ for all j and k

Or equally:

$$ \sum_i v_i x_{ij} - \sum_i v_i x_{ik} = 0$$

In [99]:
from itertools import combinations
for k, l in combinations(range(m), r=2):
    bqm.add_linear_equality_constraint(
    [(x[i, k], values[i]) for i in range(n)] + [(x[i, l], -values[i]) for i in range(n)],
    constant=0,
    lagrange_multiplier=10)

Add a constraint to make sure each value is assign to exactly one set

In [100]:
for i in range(n):
    bqm.add_linear_equality_constraint(
    [(x[i, k], 1.0) for k in range(m)],
    constant=-1.0,
    lagrange_multiplier=10)

Solve using one of the solvers. You may have to run it a few times.

In [101]:
from neal import SimulatedAnnealingSampler

res = SimulatedAnnealingSampler().sample(bqm, num_reads=100, num_sweeps=1000).truncate(5)
print(res)

  ('x_0', 0) ('x_0', 1) ('x_0', 2) ('x_1', 0) ... ('x_8', 2) energy num_oc.
0          1          0          0          1 ...          1    0.0       1
1          1          0          0          0 ...          1   10.0       1
2          0          0          1          0 ...          0   10.0       1
3          0          1          0          1 ...          0   10.0       1
4          0          0          1          0 ...          0   10.0       1
['BINARY', 5 rows, 5 samples, 27 variables]


# Result

In [102]:
sample = res.first.sample

print(sum(values))
for k in range(m):
    set1 = [values[i] for (i, l) in x if sample[x[i, l]] if k == l]
    print(sum(set1), set1)

36
12 [7, 2, 3]
12 [3, 1, 8]
12 [1, 2, 9]
